In [1]:
import numpy as np
import pandas as pd
import tensorflow  as tf

In [2]:
images_ds=tf.data.Dataset.list_files("images/*/*", shuffle=False)

In [3]:
for i in images_ds.take(5):
    print(i.numpy())

b'images\\cat\\cat.1.jpg'
b'images\\cat\\cat.10.jpg'
b'images\\cat\\cat.100.jpg'
b'images\\cat\\cat.101.jpg'
b'images\\cat\\cat.102.jpg'


In [4]:
image_count=len(images_ds)
image_count

492

In [5]:
images_ds=images_ds.shuffle(image_count)
for i in images_ds.take(5):
    print(i.numpy())

b'images\\dog\\dog.121.jpg'
b'images\\cat\\cat.57.jpg'
b'images\\cat\\cat.189.jpg'
b'images\\cat\\cat.51.jpg'
b'images\\cat\\cat.7.jpg'


In [6]:
class_names=["cat", "dog"]

In [7]:
train_size=int(image_count*0.8)

In [8]:
train_set=images_ds.take(train_size)
test_set=images_ds.skip(train_size)

In [9]:
len(train_set)

393

In [10]:
len(test_set)

99

In [11]:
import os
def get_label(file_path):
#     s=file_path.split("\\")[-2]

    s=tf.strings.split(file_path, os.path.sep)[-2]
    return s

In [12]:
print(get_label('images\\dog\\dog.121.jpg').numpy())

b'dog'


In [13]:
def process_image(file_path):
    label=get_label(file_path)
    img=tf.io.read_file(file_path)
    img=tf.image.decode_jpeg(img)
    img=tf.image.resize(img, [128,128])
    return img, label


In [14]:
process_image('images\\dog\\dog.121.jpg')

(<tf.Tensor: shape=(128, 128, 3), dtype=float32, numpy=
 array([[[136.89307 , 133.34619 , 119.98682 ],
         [104.057495,  97.057495,  78.987305],
         [135.97119 , 133.39868 , 114.71619 ],
         ...,
         [162.76343 , 169.5603  , 167.0503  ],
         [175.8822  , 179.47583 , 186.1637  ],
         [172.91125 , 186.03113 , 186.22644 ]],
 
        [[ 97.51538 , 100.53088 ,  86.6637  ],
         [ 97.07556 ,  94.90369 ,  76.770874],
         [144.98193 , 144.51318 , 125.04443 ],
         ...,
         [164.13696 , 170.87854 , 160.14722 ],
         [177.34521 , 181.51709 , 174.72803 ],
         [172.65674 , 181.24268 , 173.7583  ]],
 
        [[127.70068 , 138.74756 , 123.79224 ],
         [141.85046 , 144.17859 , 125.49768 ],
         [157.34875 , 156.34875 , 135.60815 ],
         ...,
         [133.74695 , 138.71729 , 117.2251  ],
         [140.15503 , 144.13159 , 120.99097 ],
         [128.50879 , 131.4469  , 114.3916  ]],
 
        ...,
 
        [[160.81677 , 144.81677 

In [15]:
train_set=train_set.map(process_image)

In [16]:
def scale_image(image, label):
    return image/255, label
    

In [17]:
train_set=train_set.map(scale_image)

In [18]:
test_set=test_set.map(process_image).map(scale_image)

In [19]:
for img, label in test_set.take(3):
    print("Image:",img)
    print("Label: ", label)
    

Image: tf.Tensor(
[[[0.2982709  0.3374866  0.3727807 ]
  [0.23920994 0.29803348 0.32548445]
  [0.29950023 0.3661669  0.39361787]
  ...
  [0.2435317  0.35333562 0.4082376 ]
  [0.2586933  0.36849725 0.4233992 ]
  [0.16785194 0.27765587 0.33255783]]

 [[0.35808057 0.39729625 0.43259037]
  [0.37198988 0.43081343 0.4582644 ]
  [0.30458793 0.3712546  0.39870557]
  ...
  [0.33587432 0.44567823 0.5005802 ]
  [0.27943283 0.38923675 0.4441387 ]
  [0.26552734 0.37533125 0.43023324]]

 [[0.4471584  0.48637408 0.5216682 ]
  [0.37431258 0.4331361  0.46058708]
  [0.37481618 0.44148284 0.46893382]
  ...
  [0.41228744 0.5220914  0.57699335]
  [0.33460096 0.44440487 0.49930683]
  [0.28455308 0.394357   0.44925895]]

 ...

 [[0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  ...
  [0.02432598 0.05177696 0.02432598]
  [0.02432598 0.05177696 0.02432598]
  [0.02432598 0.05177696 0.02432598]]

 [[0.         0.         0.        ]
  [0.         0.   

In [20]:
from tensorflow import keras
from tensorflow.keras import models, layers

In [21]:
cnn=models.Sequential([
    layers.Conv2D(filters=32, kernel_size=3, input_shape=(128, 128, 3), activation="relu"),
    layers.MaxPool2D(pool_size=2, strides=2),
    layers.Conv2D(filters=32, kernel_size=3, activation="relu"),
    layers.MaxPool2D(pool_size=2, strides=2),
    layers.Flatten(),
    layers.Dense(units=100, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(units=50, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(units=1, activation="sigmoid")
])

In [22]:
cnn.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [23]:
def classifier(text):
    if b'cat' in text:
        return 0
    else:
        return 1

In [24]:
x_train, y_train=[], []
for img , label in train_set:
    x_train.append(img)
    y_train.append(classifier(label.numpy()))

In [25]:
y_train[:5]

[0, 0, 1, 0, 1]

In [26]:
x_test, y_test=[],[]
for img , label in test_set:
    x_test.append(img)
    y_test.append(classifier(label.numpy()))

In [27]:
y_test[:5]

[1, 1, 0, 0, 1]

In [28]:
x_train=np.asarray(x_train)
x_test=np.asarray(x_test)
y_train=np.asarray(y_train)
y_test=np.asarray(y_test)

In [29]:
cnn.fit(x_train, y_train , epochs=15)

Epoch 1/15
13/13 [==============================] - 9s 393ms/step - loss: 0.8636 - accuracy: 0.5369
Epoch 2/15
13/13 [==============================] - 5s 386ms/step - loss: 0.6953 - accuracy: 0.5115
Epoch 3/15
13/13 [==============================] - 6s 441ms/step - loss: 0.6897 - accuracy: 0.5649
Epoch 4/15
13/13 [==============================] - 6s 440ms/step - loss: 0.6873 - accuracy: 0.5318
Epoch 5/15
13/13 [==============================] - 6s 427ms/step - loss: 0.6879 - accuracy: 0.5267
Epoch 6/15
13/13 [==============================] - 6s 485ms/step - loss: 0.6642 - accuracy: 0.6132
Epoch 7/15
13/13 [==============================] - 5s 416ms/step - loss: 0.6595 - accuracy: 0.6260
Epoch 8/15
13/13 [==============================] - 5s 408ms/step - loss: 0.6100 - accuracy: 0.6514
Epoch 9/15
13/13 [==============================] - 5s 400ms/step - loss: 0.5756 - accuracy: 0.7328
Epoch 10/15
13/13 [==============================] - 5s 378ms/step - loss: 0.4895 - accuracy: 0.8142

In [32]:
y_pre= cnn.predict(x_test)

4/4 [==============================] - 0s 54ms/step


In [33]:
y_pre[:5]

array([[0.8004052 ],
       [0.99701256],
       [0.46241423],
       [0.2807061 ],
       [0.9687217 ]], dtype=float32)

In [34]:
y_pred=[]
for i in y_pre:
    if i>0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [35]:
from sklearn.metrics import classification_report 
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.92      0.93        52
           1       0.92      0.94      0.93        47

    accuracy                           0.93        99
   macro avg       0.93      0.93      0.93        99
weighted avg       0.93      0.93      0.93        99

